In order to optimize our inventory, we would like to know which films will be rented next month and we are asked to create a model to predict it.

Firstly we'll connect to the **sakila** database:

In [3]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import getpass  # To get the password without showing the input
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [26]:
password = getpass.getpass('Get password')
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

1. Create a query or queries to extract the information you think may be relevant for building the prediction model. It should include some film features and some rental features.

We might want to extract the **rental** and **film** tables to see what type of films are most rented by customer and stock appropriately.

In [4]:
query = '''SELECT * 
FROM rental 
LEFT JOIN inventory 
ON inventory.inventory_id = rental.inventory_id
LEFT JOIN film 
ON inventory.film_id = film.film_id'''

2. Read the data into a Pandas dataframe.

In [5]:
data = pd.read_sql_query(query, engine)
data.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update,inventory_id,film_id,store_id,...,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53,367,80,1,...,2006,1,None,7,2.99,148,21.99,G,Trailers,2006-02-15 05:03:42
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53,1525,333,2,...,2006,1,None,7,2.99,126,16.99,R,"Trailers,Behind the Scenes",2006-02-15 05:03:42
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53,1711,373,2,...,2006,1,None,7,2.99,156,14.99,G,"Trailers,Behind the Scenes",2006-02-15 05:03:42
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53,2452,535,1,...,2006,1,None,6,0.99,181,21.99,R,"Trailers,Behind the Scenes",2006-02-15 05:03:42
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53,2079,450,2,...,2006,1,None,5,2.99,84,29.99,NC-17,Trailers,2006-02-15 05:03:42


In [6]:
# Check columns
data.columns

Index(['rental_id', 'rental_date', 'inventory_id', 'customer_id',
       'return_date', 'staff_id', 'last_update', 'inventory_id', 'film_id',
       'store_id', 'last_update', 'film_id', 'title', 'description',
       'release_year', 'language_id', 'original_language_id',
       'rental_duration', 'rental_rate', 'length', 'replacement_cost',
       'rating', 'special_features', 'last_update'],
      dtype='object')

In [7]:
# Drop duplicate columns
to_drop = ['film_id', 'inventory_id', 'last_update']
data.drop(to_drop, axis=1, inplace=True)

3. Analyze extracted features and transform them. You may need to encode some categorical variables, or scale numerical variables.

In [8]:
# Remove irrelevant features
to_drop = ['store_id', 'staff_id', 'customer_id', 'rental_id', 'title', 'description']
data.drop(to_drop, axis=1, inplace=True)

In [9]:
data.head()

,rental_date,return_date,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features
0,2005-05-24 22:53:30,2005-05-26 22:04:30,2006,1,None,7,2.99,148,21.99,G,Trailers
1,2005-05-24 22:54:33,2005-05-28 19:40:33,2006,1,None,7,2.99,126,16.99,R,"Trailers,Behind the Scenes"
2,2005-05-24 23:03:39,2005-06-01 22:12:39,2006,1,None,7,2.99,156,14.99,G,"Trailers,Behind the Scenes"
3,2005-05-24 23:04:41,2005-06-03 01:43:41,2006,1,None,6,0.99,181,21.99,R,"Trailers,Behind the Scenes"
4,2005-05-24 23:05:21,2005-06-02 04:33:21,2006,1,None,5,2.99,84,29.99,NC-17,Trailers


In [19]:
# Check unique values across columns
def check_unique_values(df):
    single_value_columns = []
    for column in df:
        if len(df[column].unique()) == 1:
            single_value_columns.append(column)
    return single_value_columns

check_unique_values(data)

[]

In [12]:
# Remove release_year, language_id, and original_language_id
to_drop = ['release_year', 'language_id', 'original_language_id']
data.drop(to_drop, axis=1, inplace=True)

In [17]:
# Check null values in data
def check_null_values(df, threshold=75):
    nulls_percentage = {}
    for column in df.columns:
        number_of_nulls = df[column].isna().sum()
        null_percentage = round(number_of_nulls * 100 / df.shape[0], 1)
        if null_percentage >= threshold:
            nulls_percentage[column] = null_percentage
    return nulls_percentage

check_null_values(data)

{}

In [25]:
# Check maximum null values in a dataframe column
def max_nulls(df):
    nulls_percentage = []
    for column in df.columns:
        number_of_nulls = df[column].isna().sum()
        null_percentage = round(number_of_nulls * 100 / df.shape[0], 1)
        nulls_percentage.append(null_percentage)
    return max(nulls_percentage)

max_nulls(data)

0.0

In [24]:
# Check null values per row
def max_nulls_rows(df):
    nulls_percentage = []
    for index in df.index:
        number_of_nulls = df.loc[index,].isna().sum()
        null_percentage = round(number_of_nulls * 100 / df.shape[1], 1)
        nulls_percentage.append(null_percentage)
    return max(nulls_percentage)

max_nulls_rows(data)

0.0

In [23]:
# Check number of null values
for col in data.columns:
    if data[col].isna().sum() != 0:
        print(col, ":", round((data[col].isna().sum()) * 100 / data.shape[0], 0),"%")

In [22]:
# As return_date is the only column which still has NaN values, we can remove
# drop all of the rows, as we won't lose more than 1% of data
data = data.dropna()

4. Create a query to get the list of films and a boolean indicating if it was rented last month. This would be our target variable.

In [13]:
# Check the column data type
data.rental_date.dtype

dtype('<M8[ns]')

In [14]:
# Check the entries data type
type(data.rental_date[0])

pandas._libs.tslibs.timestamps.Timestamp

In [15]:
# Check the last month a movie was rented
max(data.rental_date)

Timestamp('2006-02-14 15:16:03')

In [16]:
# We'll use the last month (February 2006) as the reference month for this exercise
data['last_month_rental'] = data['rental_date'].apply(lambda x: True if \
    str(x.to_period('M')) == '2006-02' else False)

data.tail()

,rental_date,return_date,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_month_rental
15635,2005-08-23 22:25:26,2005-08-25 23:54:26,5,0.99,67,23.99,R,"Trailers,Behind the Scenes",False
15636,2005-08-23 22:26:47,2005-08-27 18:02:47,6,0.99,78,28.99,PG-13,"Commentaries,Behind the Scenes",False
15637,2005-08-23 22:42:48,2005-08-25 02:48:48,4,0.99,122,15.99,R,"Commentaries,Behind the Scenes",False
15638,2005-08-23 22:43:07,2005-08-31 21:33:07,4,4.99,151,28.99,PG-13,"Trailers,Deleted Scenes",False
15639,2005-08-23 22:50:12,2005-08-30 01:01:12,4,0.99,105,25.99,G,Trailers,False


5. Create a logistic regression model to predict this variable from the cleaned data.

In [ ]:
X = data.select_dtypes(include = np.number)

transformer = StandardScaler().fit(X)
x_normalized = transformer.transform(X)
x = pd.DataFrame(x_normalized)
x.columns = X.columns
x.head()

In [ ]:
cat = data.select_dtypes(include = np.object)
cat = cat.drop(['status'], axis=1)
categorical = pd.get_dummies(cat, columns=['type', 'operation', 'k_symbol', 'duration'])

In [ ]:
y = data['status'] # Target appart
X = np.concatenate((x, categorical), axis=1) # Numerical and dummies from categoricals

In [ ]:
# We separate training and testing datasets and correspondant targets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

In [ ]:
# We traing the training dataset using Logistic Regression (multiclass classification)
classification = LogisticRegression(random_state=0, multi_class='multinomial').fit(X_train, y_train)

In [ ]:
# We predict on the testing dataset
predictions = classification.predict(X_test)

6. Evaluate the results.

In [ ]:
pd.Series(predictions).value_counts()

In [ ]:
y_test.value_counts()

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, predictions)
sns.heatmap(cm, annot=True,fmt='g')
plt.subplot().xaxis.set_ticklabels(["A","B","C","D"])
plt.subplot().yaxis.set_ticklabels(["A","B","C","D"])